In [298]:

import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score



In [299]:

# Charger les données annotées avec 'commentaire' et 'sentiment' (positif, négatif, neutre)
data=pd.read_csv('C:/Users/Lenovo/Desktop/cloud compuing/PFA/code/train_dataset3.csv')


In [300]:
data

,id,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,...,source,retweets,favorites,is_retweet,cleaned_text_textblob,polarity,subjectivity,Sentiment_textBlob,cleaned_text_vader,Sentiment_vader
0,1365695382076854280,Seeji Sundarakshan,Mumbai & Kerala.,"interest in geopolitics, economics, independen...",2016-09-04 09:39:11,608,277,27106,False,2021-02-27 16:08:47,...,Twitter Web App,0,1,False,consolation good number sputnikv instead going...,0.70,0.60,Positive,the only consolation is that good number of sp...,Positive
1,1379853509902934025,Tim Niblett,Burlington ON and Hudson FL,NaN,2011-03-28 20:43:51,88,219,5375,False,2021-04-07 17:48:07,...,Twitter for Android,0,2,False,wife got 2nd moderna shot yesterday florida . ...,0.25,0.25,Positive,my wife and i both got our 2nd moderna shots y...,Positive
2,1446041875815743491,KAS Sub-Saharan Africa,"Berlin, Germany",Official account Sub-Saharan Africa Dept. @KAS...,2021-05-31 14:19:29,312,195,412,False,2021-10-07 09:17:04,...,Twitter Web App,0,3,False,"following author study tom bayes , china prese...",0.00,0.05,Neutral,"following the author of the study tom bayes, c...",Negative
3,1408683429072232448,CowinBangalore,NaN,Follow us to get update as soon as 18 plus vac...,2021-05-17 18:56:45,1102,2,57,False,2021-06-26 07:07:56,...,cowin_vaccine_app,0,0,False,"560011 , age 45 41d1 , 276d2 slot , covaxin 30...",0.00,0.00,Neutral,"560011, age 45 41d1, 276d2 slots, covaxin on 3...",Neutral
4,1410637131131805705,🐝Lisa🐝,Jersey,A happy Bean living in a beautiful island,2009-05-02 15:15:26,41,489,1940,False,2021-07-01 16:31:15,...,Twitter for iPhone,0,0,False,happy 2nd vaccine day ! ! ! moderna,0.40,0.50,Positive,happy 2nd vaccine day to me!!! moderna,Positive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86810,1365360554186072070,James,"Leeds, England",$ocgn #ocgn #dogecoin,2020-06-11 20:27:59,83,359,3943,False,2021-02-26 17:58:17,...,Twitter for iPhone,6,12,False,"ocgn stock smiling_face_with_sunglasses , huge...",0.20,0.95,Positive,ocgn is the stock smiling_face_with_sunglasses...,Positive
86811,1441160179035381762,CGTN,"Beijing, China",CGTN is an international media organization. I...,2013-01-24 03:18:59,13466879,74,167,True,2021-09-23 21:58:56,...,TweetDeck,5,10,False,noted modis visit go current you.s . fda cdc q...,0.00,0.40,Neutral,some have noted modis visit goes against curre...,Neutral
86812,1423247259215405058,CoWIN Blore 18-44,"Bengaluru, India",Get instant alerts for Free/Paid slots in BBMP...,2021-06-29 09:33:13,19,0,1,False,2021-08-05 11:39:23,...,CowinAlertsBot,0,0,False,pincode 560027 covaxin dose 2 50 slot age 18 a...,0.00,0.00,Neutral,pincode 560027 covaxin dose 2 50 slots age 18 ...,Neutral
86813,1415288394859487233,Owl 🦉,NaN,follow for updates on 18-44 vaccination availa...,2021-05-26 19:13:22,147,0,1,False,2021-07-14 12:33:42,...,Cowin Vaccination Availability,0,0,False,"180342 400602 , 16072021 , 61 dose2 highway ho...",0.00,0.00,Neutral,"180342 400602, 16072021, 61 dose2 at highway h...",Neutral


In [301]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from gensim.models import Word2Vec
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re
nltk.download('punkt')
nltk.download('stopwords')

# Obtenir la liste des stop words en anglais
stop_words = set(stopwords.words('english'))

# Mots importants à conserver
mots_importants = {
    "not", "no", "nor", "isn't", "aren't", "wasn't", "weren't", "don't", "doesn't", 
    "didn't", "won't", "wouldn't", "shan't", "can't", "couldn't", "haven't", 
    "hasn't", "hadn't", "mustn't", "mightn't", "needn't", "shouldn't"
}

# Créer une nouvelle liste de stop words en retirant les mots importants
stop_words_personnalises = stop_words - mots_importants

# Fonction de nettoyage du texte
def nettoyer_texte(texte):
    # Conversion en minuscules
    texte = texte.lower()
    
    # Suppression des liens URL
    texte = re.sub(r'http\S+|www\S+|https\S+', '', texte, flags=re.MULTILINE)

    # Suppression des mentions
    texte = re.sub(r'@\w+', '', texte)

    # Suppression du symbole # et garder le mot après
    texte = re.sub(r'#(\w+)', r'\1', texte)

    # Suppression des espaces supplémentaires
    texte = re.sub(r'\s+', ' ', texte).strip()

    # Garder lettres, chiffres, et les ponctuations importantes: ! ? . ,
    texte = re.sub(r'[^\w\s!?.,]', '', texte)

    # Tokenisation
    tokens = word_tokenize(texte)

    # Suppression des stop words personnalisés
    tokens = [mot for mot in tokens if mot not in stop_words_personnalises]

    return tokens

# Nettoyer les commentaires et les tokeniser
data['cleaned_comments'] = data['text'].apply(nettoyer_texte)
print(data['cleaned_comments'].str.strip().eq('').sum())  # Vérifiez les chaînes vides
# Supprimer les lignes où la colonne 'text' est nulle ou vide
data = data[~data['cleaned_comments'].isnull() & (data['cleaned_comments'].str.strip() != '')]


# Entraîner un modèle Word2Vec
sentences = data['cleaned_comments'].tolist()  # Liste de listes de mots (chaque tweet est une liste de tokens)
word2vec_model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)

# Fonction pour obtenir le vecteur moyen d'un tweet
def vectorize_tweet(tweet, model):
    word_vectors = [model.wv[word] for word in tweet if word in model.wv]
    if word_vectors:
        return np.mean(word_vectors, axis=0)
    else:
        # Retourner un vecteur nul si aucun mot du tweet n'est dans le modèle
        return np.zeros(model.vector_size)

# Appliquer la vectorisation sur les tweets
X_vectorized = np.array([vectorize_tweet(tweet, word2vec_model) for tweet in data['cleaned_comments']])


# Mapping des labels
label_mapping = {'Negative': 0, 'Neutral': 1, 'Positive': 2}


y = data['Sentiment_vader'].map(label_mapping)


# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X_vectorized, y, test_size=0.2, random_state=42)

# Initialiser et entraîner le modèle KNN
k = 5  # Choisissez une valeur de K
knn = KNeighborsClassifier(n_neighbors=k)
knn.fit(X_train, y_train)

# Prédictions sur l'ensemble de test
y_test_pred = knn.predict(X_test)

# Calcul de l'accuracy pour l'entraînement et la validation
train_accuracy = knn.score(X_train, y_train)
test_accuracy = knn.score(X_test, y_test)

# Affichage des résultats
print(f'Accuracy on training set: {train_accuracy:.4f}')
print(f'Accuracy on test set: {test_accuracy:.4f}')

# Évaluation du modèle
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_test_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_test_pred, target_names=['Negative', 'Neutral', 'Positive']))


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


0
Accuracy on training set: 0.7863
Accuracy on test set: 0.6864
Confusion Matrix:
[[4438  706  772]
 [1268 3922  460]
 [1584  655 3558]]

Classification Report:
              precision    recall  f1-score   support

    Negative       0.61      0.75      0.67      5916
     Neutral       0.74      0.69      0.72      5650
    Positive       0.74      0.61      0.67      5797

    accuracy                           0.69     17363
   macro avg       0.70      0.69      0.69     17363
weighted avg       0.70      0.69      0.69     17363



#### tester KNN 

In [302]:

# Charger les données annotées avec 'commentaire' et 'sentiment' (positif, négatif, neutre)
new_data=pd.read_csv('C:/Users/Lenovo/Desktop/cloud compuing/PFA/code/test_dataset3.csv')


In [303]:
# Étape 1 : Nettoyer les nouveaux commentaires
new_data['cleaned_comments'] = new_data['text'].apply(nettoyer_texte)

# Étape 2 : Vectoriser les nouveaux commentaires
def vectorize_tweet(tweet, model):
    word_vectors = [model.wv[word] for word in tweet if word in model.wv]
    if word_vectors:
        return np.mean(word_vectors, axis=0)
    else:
        return np.zeros(model.vector_size)

# Appliquer la vectorisation sur les nouveaux tweets
X_new_vectorized = np.array([vectorize_tweet(tweet, word2vec_model) for tweet in new_data['cleaned_comments']])

# Prédire les classes pour les nouveaux commentaires
y_new_pred = knn.predict(X_new_vectorized)



In [304]:

# Mapping des labels
label_mapping = {'Negative': 0, 'Neutral': 1, 'Positive': 2}



In [305]:
y_new_pred

array([0, 1, 1, ..., 2, 1, 2], dtype=int64)

In [306]:


# Mapping des labels
sentiment=nouveaux_donnees['Sentiment_vader'].map(label_mapping)


In [307]:
sentiment

0        0
1        1
2        1
3        2
4        1
        ..
21699    2
21700    1
21701    2
21702    1
21703    2
Name: Sentiment_vader, Length: 21704, dtype: int64

In [308]:

accuracy = accuracy_score(sentiment, y_new_pred)
print(f"\nAccuracy sur le nouvel ensemble de données : {accuracy * 100:.2f}%")



Accuracy sur le nouvel ensemble de données : 69.42%


In [310]:
# 10. Afficher la matrice de confusion
conf_matrix = confusion_matrix(sentiment, y_new_pred)
print("\nMatrice de confusion :")
print(conf_matrix)


Matrice de confusion :
[[5548  826 1004]
 [1562 5021  580]
 [1929  736 4498]]


In [311]:
# 11. Afficher le rapport de classification
print("\nRapport de classification :")
print(classification_report(sentiment, y_new_pred))


Rapport de classification :
              precision    recall  f1-score   support

           0       0.61      0.75      0.68      7378
           1       0.76      0.70      0.73      7163
           2       0.74      0.63      0.68      7163

    accuracy                           0.69     21704
   macro avg       0.71      0.69      0.70     21704
weighted avg       0.70      0.69      0.70     21704

